In [1]:
from __future__ import division
from __future__ import print_function

import os, sys
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import random
import logging
import config
import torch.optim as optim
import numpy as np
import pandas as pd

# NEURAL NETWORK MODULES/LAYERS
from model import *
from dataset import Dataset
# METRICS CLASS FOR EVALUATION
from metrics import Metrics
# CONFIG PARSER
from config import parse_args
# TRAIN AND TEST HELPER FUNCTIONS
from trainer import Trainer
import glob
from tqdm import tqdm

import torchvision
from torchvision import models


# MAIN BLOCK
# def main():
# global args
# args = parse_args()
# global logger


In [2]:
class config():
    def __init__(self):
        self.data='./data'
        self.save='checkpoints/'
        self.expname='test'
        self.expno=0
        self.input_dim=300
        self.pretrained_model='vgg16'
        self.num_classes=16
        # training arguments
        self.epochs=1000
        self.batchsize=2
        self.lr=0.001
        self.wd=1e-4
        self.sparse=False
        self.optim='adam'
        # miscellaneous options
        self.seed=123
        self.cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.cuda else "cpu")
#         cuda_parser = parser.add_mutually_exclusive_group(required=False)
#         cuda', dest='cuda', action='store_true')
#         cuda_parser.add_argument('--no-cuda', dest='cuda', action='store_false')
#         parser.set_defaults(cuda=True)
        

In [3]:
args = config()
args.batchsize

2

In [4]:
logger = logging.getLogger('dc_log')
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter("[%(asctime)s] %(levelname)s:%(name)s:%(message)s")
# file logger
fh = logging.FileHandler(os.path.join(args.save, args.expname)+'.log', mode='w')
fh.setLevel(logging.INFO)
fh.setFormatter(formatter)
logger.addHandler(fh)
# console logger
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)
logger.addHandler(ch)
# argument validation
if args.sparse and args.wd!=0:
    logger.error('Sparsity and weight decay are incompatible, pick one!')
    exit()
logger.debug(args)
torch.manual_seed(args.seed)
random.seed(args.seed)

if args.cuda:
    torch.cuda.manual_seed(args.seed)
    torch.backends.cudnn.benchmark = True

if not os.path.exists(args.save):
    os.makedirs(args.save)



# initialize model, criterion/loss_function, optimizer
if args.pretrained_model == 'vgg16':
    pretrained_model = models.vgg16(pretrained=True)

# Freeze training for all layers
for param in pretrained_model.features.parameters():
    param.requires_grad = False

for param in pretrained_model.classifier.parameters():
    param.requires_grad = False


model = DocClassification(args, pretrained_model)
criterion = nn.CrossEntropyLoss(reduction='sum')

parameters = filter(lambda p: p.requires_grad, model.parameters())

if args.cuda:
    model.cuda(), criterion.cuda()

if args.optim=='adam':
    optimizer   = optim.Adam(parameters, lr=args.lr, weight_decay=args.wd)
elif args.optim=='adagrad':
    optimizer   = optim.Adagrad(parameters, lr=args.lr, weight_decay=args.wd)
elif args.optim=='sgd':
    optimizer   = optim.SGD(parameters, lr=args.lr, weight_decay=args.wd)
elif args.optim == 'adadelta':
    optimizer = optim.Adadelta(parameters, lr=args.lr, weight_decay=args.wd)

metrics = Metrics(args.num_classes)

# create trainer object for training and testing
trainer = Trainer(args, model, criterion, optimizer)





[2019-10-20 17:57:18,001] DEBUG:dc_log:<__main__.config object at 0x7f5f51f23e50>


In [5]:
train_dir = glob.glob(os.path.join(args.data,'train/holistic/*.jpg'))
dev_dir = glob.glob(os.path.join(args.data,'val/holistic/*.jpg'))
test_dir = glob.glob(os.path.join(args.data,'test/holistic/*.jpg'))

print(len(train_dir), len(dev_dir), len(test_dir))

# 'data/labels/train.csv'
train_dataset = Dataset('data/labels/train.csv','train', train_dir)
dev_dataset = Dataset('data/labels/val.csv','val',dev_dir)
test_dataset = Dataset('data/labels/test.csv','test', test_dir)


logger.debug('==> Size of train data   : %d ' % len(train_dataset))
logger.debug('==> Size of val data   : %d ' % len(dev_dataset))
logger.debug('==> Size of test data   : %d ' % len(test_dataset))

train_idx = list(np.arange(len(train_dataset)))
dev_idx = list(np.arange(len(dev_dataset)))
test_idx = list(np.arange(len(test_dataset)))

best = float('inf')
columns = ['ExpName','ExpNo', 'Epoch', 'Loss','Accuracy']
results = []
early_stop_count = 0

[2019-10-20 17:57:20,417] DEBUG:dc_log:==> Size of train data   : 80000 
[2019-10-20 17:57:20,418] DEBUG:dc_log:==> Size of val data   : 16000 
[2019-10-20 17:57:20,418] DEBUG:dc_log:==> Size of test data   : 16000 


80000 16000 16000


In [6]:

for epoch in range(args.epochs):

    train_loss = 0.0
    dev_loss = 0.0
    test_loss = 0.0

    train_predictions = []
    train_labels = []

    dev_predictions = []
    dev_labels = []

    test_predictions = []
    test_labels = []

    

    batch_train_data = list(np.random.randint(0, len(train_idx), args.batchsize))
    batch_dev_data = list(np.random.randint(0, len(dev_idx), args.batchsize))
    batch_test_data = list(np.random.randint(0, len(test_idx), args.batchsize))
    
#     for batch in tqdm(batch_train_data, desc='Training batches..'):
    batch_holistic, _, _, _, _, batch_labels = train_dataset[batch_train_data]
    _ = trainer.train_holistic(batch_holistic, batch_labels)

#     for batch in tqdm(batch_train_data, desc='Training batches..'):
    train_batch_holistic, _, _, _, _, train_batch_labels = train_dataset[batch_train_data]
    train_batch_loss, train_batch_predictions, train_batch_labels = trainer.test_holistic(train_batch_holistic, train_batch_labels)

#         train_predictions.append(train_batch_predictions)
#         train_labels.append(train_batch_labels)
#         train_loss = train_loss + train_batch_loss

    train_accuracy = metrics.accuracy(train_batch_predictions, train_batch_labels)

#     for batch in tqdm(batch_dev_data, desc='Dev batches..'):
    dev_batch_holistic, _, _, _, _, dev_batch_labels = dev_dataset[batch_dev_data]
    dev_batch_loss, dev_batch_predictions, dev_batch_labels = trainer.test_holistic(dev_batch_holistic, dev_batch_labels)

#         dev_predictions.append(dev_batch_predictions)
#         dev_labels.append(dev_batch_labels)
#         dev_loss = dev_loss + dev_batch_loss

    dev_accuracy = metrics.accuracy(dev_batch_predictions, dev_batch_labels)

#     for batch in tqdm(batch_test_data, desc='Test batches..'):
    test_batch_holistic, _, _, _, _, test_batch_labels = test_dataset[batch_test_data]
    test_batch_loss, test_batch_predictions, test_batch_labels = trainer.test_holistic(test_batch_holistic, test_batch_labels)

#         test_predictions.append(test_batch_predictions)
#         test_labels.append(test_batch_labels)
#         test_loss = test_loss + test_batch_loss

    test_accuracy = metrics.accuracy(test_batch_predictions, test_batch_labels)

    logger.info('==> Training Epoch {}, \
                    \nLoss: {}, \
                    \nAccuracy: {}'.format(epoch + 1, train_batch_loss, train_accuracy))
    logger.info('==> Dev Epoch {}, \
                    \nLoss: {}, \
                    \nAccuracy: {}'.format(epoch + 1, dev_batch_loss, dev_accuracy))

    logger.info('==> Test Epoch {}, \
                    \nLoss: {}, \
                    \nAccuracy: {}'.format(epoch + 1, test_batch_loss, test_accuracy))
    #quit()
    results.append((args.expname, \
                    args.expno, \
                    epoch+1, \
                    test_batch_loss, \
                    test_accuracy))

    if best > test_batch_loss:
        best = test_batch_loss
        checkpoint = {'model': trainer.model.state_dict(), 'optim': trainer.optimizer,
                      'loss': test_batch_loss, 'accuracy': test_accuracy,
                      'args': args, 'epoch': epoch }
        logger.debug('==> New optimum found, checkpointing everything now...')
        torch.save(checkpoint, '%s.pt' % os.path.join(args.save, args.expname))
        #np.savetxt("test_pred.csv", test_pred.numpy(), delimiter=",")
    else:
        early_stop_count = early_stop_count + 1

        if early_stop_count == 20:
            quit()



[2019-10-20 17:57:21,525] INFO:dc_log:==> Training Epoch 1,                     
Loss: 1.6120359897613525,                     
Accuracy: 0.5
[2019-10-20 17:57:21,526] INFO:dc_log:==> Dev Epoch 1,                     
Loss: 22.595050811767578,                     
Accuracy: 0.0
[2019-10-20 17:57:21,527] INFO:dc_log:==> Test Epoch 1,                     
Loss: 13.483207702636719,                     
Accuracy: 0.0
[2019-10-20 17:57:21,528] DEBUG:dc_log:==> New optimum found, checkpointing everything now...
[2019-10-20 17:57:22,319] INFO:dc_log:==> Training Epoch 2,                     
Loss: 4.660411357879639,                     
Accuracy: 0.5
[2019-10-20 17:57:22,320] INFO:dc_log:==> Dev Epoch 2,                     
Loss: 45.87076950073242,                     
Accuracy: 0.0
[2019-10-20 17:57:22,321] INFO:dc_log:==> Test Epoch 2,                     
Loss: 57.548377990722656,                     
Accuracy: 0.0
[2019-10-20 17:57:22,401] INFO:dc_log:==> Training Epoch 3,               

[2019-10-20 17:57:27,055] INFO:dc_log:==> Training Epoch 20,                     
Loss: 32.3543586730957,                     
Accuracy: 0.5
[2019-10-20 17:57:27,056] INFO:dc_log:==> Dev Epoch 20,                     
Loss: 39.99595260620117,                     
Accuracy: 0.0
[2019-10-20 17:57:27,056] INFO:dc_log:==> Test Epoch 20,                     
Loss: 5.187291145324707,                     
Accuracy: 0.5
[2019-10-20 17:57:27,134] INFO:dc_log:==> Training Epoch 21,                     
Loss: 46.928977966308594,                     
Accuracy: 0.0
[2019-10-20 17:57:27,135] INFO:dc_log:==> Dev Epoch 21,                     
Loss: 50.24003219604492,                     
Accuracy: 0.0
[2019-10-20 17:57:27,136] INFO:dc_log:==> Test Epoch 21,                     
Loss: 44.10085678100586,                     
Accuracy: 0.0
[2019-10-20 17:57:27,215] INFO:dc_log:==> Training Epoch 22,                     
Loss: 23.78335952758789,                     
Accuracy: 0.0
[2019-10-20 17:57:27,216

[2019-10-20 17:57:28,550] INFO:dc_log:==> Test Epoch 39,                     
Loss: 44.45451354980469,                     
Accuracy: 0.0
[2019-10-20 17:57:28,626] INFO:dc_log:==> Training Epoch 40,                     
Loss: 38.5387077331543,                     
Accuracy: 0.0
[2019-10-20 17:57:28,627] INFO:dc_log:==> Dev Epoch 40,                     
Loss: 19.14975357055664,                     
Accuracy: 0.0
[2019-10-20 17:57:28,627] INFO:dc_log:==> Test Epoch 40,                     
Loss: 14.640018463134766,                     
Accuracy: 0.0
[2019-10-20 17:57:28,703] INFO:dc_log:==> Training Epoch 41,                     
Loss: 12.041622161865234,                     
Accuracy: 0.5
[2019-10-20 17:57:28,704] INFO:dc_log:==> Dev Epoch 41,                     
Loss: 2.716721534729004,                     
Accuracy: 0.5
[2019-10-20 17:57:28,705] INFO:dc_log:==> Test Epoch 41,                     
Loss: 15.713510513305664,                     
Accuracy: 0.0
[2019-10-20 17:57:28,781] 

[2019-10-20 17:57:30,098] INFO:dc_log:==> Dev Epoch 59,                     
Loss: 52.11875915527344,                     
Accuracy: 0.0
[2019-10-20 17:57:30,099] INFO:dc_log:==> Test Epoch 59,                     
Loss: 15.766944885253906,                     
Accuracy: 0.0
[2019-10-20 17:57:30,173] INFO:dc_log:==> Training Epoch 60,                     
Loss: 9.047430038452148,                     
Accuracy: 0.0
[2019-10-20 17:57:30,174] INFO:dc_log:==> Dev Epoch 60,                     
Loss: 28.112592697143555,                     
Accuracy: 0.0
[2019-10-20 17:57:30,175] INFO:dc_log:==> Test Epoch 60,                     
Loss: 46.79767608642578,                     
Accuracy: 0.0
[2019-10-20 17:57:30,253] INFO:dc_log:==> Training Epoch 61,                     
Loss: 22.109355926513672,                     
Accuracy: 0.0
[2019-10-20 17:57:30,254] INFO:dc_log:==> Dev Epoch 61,                     
Loss: 20.00628662109375,                     
Accuracy: 0.5
[2019-10-20 17:57:30,254] 

[2019-10-20 17:57:31,695] INFO:dc_log:==> Training Epoch 79,                     
Loss: 0.00044155120849609375,                     
Accuracy: 1.0
[2019-10-20 17:57:31,695] INFO:dc_log:==> Dev Epoch 79,                     
Loss: 30.548437118530273,                     
Accuracy: 0.0
[2019-10-20 17:57:31,696] INFO:dc_log:==> Test Epoch 79,                     
Loss: 123.83193969726562,                     
Accuracy: 0.0
[2019-10-20 17:57:31,793] INFO:dc_log:==> Training Epoch 80,                     
Loss: 19.744178771972656,                     
Accuracy: 0.5
[2019-10-20 17:57:31,793] INFO:dc_log:==> Dev Epoch 80,                     
Loss: 45.98589324951172,                     
Accuracy: 0.5
[2019-10-20 17:57:31,794] INFO:dc_log:==> Test Epoch 80,                     
Loss: 37.782257080078125,                     
Accuracy: 0.0
[2019-10-20 17:57:31,875] INFO:dc_log:==> Training Epoch 81,                     
Loss: 24.873477935791016,                     
Accuracy: 0.0
[2019-10-20 17

...

[2019-10-20 17:57:57,858] INFO:dc_log:==> Dev Epoch 413,                     
Loss: 8.955772399902344,                     
Accuracy: 0.5
[2019-10-20 17:57:57,859] INFO:dc_log:==> Test Epoch 413,                     
Loss: 29.078365325927734,                     
Accuracy: 0.5
[2019-10-20 17:57:57,937] INFO:dc_log:==> Training Epoch 414,                     
Loss: 2.2144291400909424,                     
Accuracy: 0.5
[2019-10-20 17:57:57,938] INFO:dc_log:==> Dev Epoch 414,                     
Loss: 16.212738037109375,                     
Accuracy: 0.0
[2019-10-20 17:57:57,939] INFO:dc_log:==> Test Epoch 414,                     
Loss: 60.98870849609375,                     
Accuracy: 0.5
[2019-10-20 17:57:58,018] INFO:dc_log:==> Training Epoch 415,                     
Loss: 0.0018582940101623535,                     
Accuracy: 1.0
[2019-10-20 17:57:58,019] INFO:dc_log:==> Dev Epoch 415,                     
Loss: 1.0013580322265625e-05,                     
Accuracy: 1.0
[2019-10-2

[2019-10-20 17:57:59,454] INFO:dc_log:==> Training Epoch 433,                     
Loss: 8.506387710571289,                     
Accuracy: 0.5
[2019-10-20 17:57:59,455] INFO:dc_log:==> Dev Epoch 433,                     
Loss: 126.58751678466797,                     
Accuracy: 0.0
[2019-10-20 17:57:59,456] INFO:dc_log:==> Test Epoch 433,                     
Loss: 39.592987060546875,                     
Accuracy: 0.5
[2019-10-20 17:57:59,534] INFO:dc_log:==> Training Epoch 434,                     
Loss: 47.87881851196289,                     
Accuracy: 0.0
[2019-10-20 17:57:59,534] INFO:dc_log:==> Dev Epoch 434,                     
Loss: 31.258195877075195,                     
Accuracy: 0.0
[2019-10-20 17:57:59,535] INFO:dc_log:==> Test Epoch 434,                     
Loss: 44.14125442504883,                     
Accuracy: 0.5
[2019-10-20 17:57:59,611] INFO:dc_log:==> Training Epoch 435,                     
Loss: 35.901771545410156,                     
Accuracy: 0.0
[2019-10-20 1

[2019-10-20 17:58:00,956] INFO:dc_log:==> Test Epoch 452,                     
Loss: 45.04774475097656,                     
Accuracy: 0.0
[2019-10-20 17:58:01,035] INFO:dc_log:==> Training Epoch 453,                     
Loss: 20.738004684448242,                     
Accuracy: 0.5
[2019-10-20 17:58:01,035] INFO:dc_log:==> Dev Epoch 453,                     
Loss: 0.0,                     
Accuracy: 1.0
[2019-10-20 17:58:01,036] INFO:dc_log:==> Test Epoch 453,                     
Loss: 77.7269287109375,                     
Accuracy: 0.5
[2019-10-20 17:58:01,111] INFO:dc_log:==> Training Epoch 454,                     
Loss: 94.2624740600586,                     
Accuracy: 0.5
[2019-10-20 17:58:01,112] INFO:dc_log:==> Dev Epoch 454,                     
Loss: 20.031694412231445,                     
Accuracy: 0.5
[2019-10-20 17:58:01,113] INFO:dc_log:==> Test Epoch 454,                     
Loss: 57.59280014038086,                     
Accuracy: 0.5
[2019-10-20 17:58:01,190] INFO:dc_l

[2019-10-20 17:58:02,529] INFO:dc_log:==> Dev Epoch 472,                     
Loss: 92.47073364257812,                     
Accuracy: 0.0
[2019-10-20 17:58:02,530] INFO:dc_log:==> Test Epoch 472,                     
Loss: 9.698763847351074,                     
Accuracy: 0.5
[2019-10-20 17:58:02,610] INFO:dc_log:==> Training Epoch 473,                     
Loss: 18.17812728881836,                     
Accuracy: 0.5
[2019-10-20 17:58:02,611] INFO:dc_log:==> Dev Epoch 473,                     
Loss: 8.034706115722656e-05,                     
Accuracy: 1.0
[2019-10-20 17:58:02,611] INFO:dc_log:==> Test Epoch 473,                     
Loss: 61.467506408691406,                     
Accuracy: 0.5
[2019-10-20 17:58:02,691] INFO:dc_log:==> Training Epoch 474,                     
Loss: 13.112143516540527,                     
Accuracy: 0.5
[2019-10-20 17:58:02,692] INFO:dc_log:==> Dev Epoch 474,                     
Loss: 49.16187286376953,                     
Accuracy: 0.0
[2019-10-20 17:5

KeyboardInterrupt: 

In [ ]:
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

In [ ]:

with open('stratified/rvl-cdip/labels/train.csv','rU') as label_file:
    X,y = [],[]
    for label in label_file.readlines():
        X.append(label.split(' ')[0])
        y.append(int(label.split(' ')[1].rstrip()))
        break
X,y

In [ ]:
df = pd.read_csv('data/stratified/rvl-cdip/labels/train.csv',sep=' ')

In [ ]:
df.head()


In [ ]:
'../stratified/rvl-cdip/images/imagese/e/j/e/eje42e00/2500126531_2500126536.tif'

In [ ]:
import dataset

In [ ]:
dataset.